In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import math
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
from pmdarima import auto_arima
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [ ]:
# Suprimir todos os warnings (não é recomendado em produção)
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('consumo.csv')

In [ ]:
df['dt_geracao'] = pd.to_datetime(df['dt_geracao'], format='%d-%m-%Y')

In [ ]:
df['ano'] = df['dt_geracao'].dt.year

In [ ]:
df['nr_valor'] = df['nr_valor'].str.replace(',', '.').astype(float)

In [ ]:
df['PreçoUnitario'] = df['nr_valor']/df['nr_quantidade']

In [ ]:
COVID = df[(df['ano']==2020)|(df['ano']==2021)]
COVID = COVID.sort_values(by=['nm_material', 'dt_geracao'])
COVID['demanda_diaria'] = -COVID['nr_qtd_estoque'].diff().where(COVID['nm_material'] == COVID['nm_material'].shift()).fillna(0)
COVID['demanda_diaria'] = COVID['demanda_diaria'].apply(lambda x: max(0, x))


#Métrica dias com dados
contagem_distintos = COVID.groupby('nm_material')['dt_geracao'].nunique().reset_index()

#Metrica Compra
CompraMedia = COVID.groupby('nm_material')['nr_quantidade'].mean().reset_index()

#Metricas para o estoque do produto
mediaEstoque = COVID.groupby('nm_material')['nr_qtd_estoque'].mean().reset_index()
minimoEstoque = COVID.groupby('nm_material')['nr_qtd_estoque'].min().reset_index()

#Metrica de Demanda diária
DemandaDiariaMedia = COVID.groupby('nm_material')['demanda_diaria'].mean().reset_index()

#MetricaPreçoProduto
PrecoUnitario = COVID.groupby('nm_material')['PreçoUnitario'].mean().reset_index()



QuantidadeDatas = pd.merge(CompraMedia,mediaEstoque,how='inner',on='nm_material')

QuantidadeDatas = pd.merge(QuantidadeDatas,minimoEstoque,on='nm_material')

QuantidadeDatas = pd.merge(QuantidadeDatas,DemandaDiariaMedia,on='nm_material')

QuantidadeDatas = pd.merge(QuantidadeDatas,PrecoUnitario,on='nm_material')

QuantidadeDatas = pd.merge(QuantidadeDatas,contagem_distintos,on='nm_material')

QuantidadeDatas = QuantidadeDatas[(QuantidadeDatas['dt_geracao']>20)]
QuantidadeDatas = QuantidadeDatas.rename(columns={'nm_material': 'Material', 'nr_quantidade': 'CompreMedia', 'nr_qtd_estoque_x': 'MediaEstoque', 'nr_qtd_estoque_y': 'MinimoEstoque'})
QuantidadeDatas['CompreMediaNorm'] = QuantidadeDatas['CompreMedia']/QuantidadeDatas['CompreMedia'].sum()
QuantidadeDatas['MediaEstoqueNorm'] = QuantidadeDatas['MediaEstoque']/QuantidadeDatas['MediaEstoque'].sum()
QuantidadeDatas['MinimoEstoqueNorm'] = QuantidadeDatas['MinimoEstoque']/QuantidadeDatas['MinimoEstoque'].sum()
QuantidadeDatas['demanda_diariaNorm'] = QuantidadeDatas['demanda_diaria']/QuantidadeDatas['demanda_diaria'].sum()
QuantidadeDatas['PreçoUnitarioNorm'] = QuantidadeDatas['PreçoUnitario']/QuantidadeDatas['PreçoUnitario'].sum()
QuantidadeDatas['dt_geracaoNorm'] = QuantidadeDatas['dt_geracao']/QuantidadeDatas['dt_geracao'].sum()
data_for_clustering = QuantidadeDatas[[ 'dt_geracaoNorm','CompreMediaNorm','MediaEstoqueNorm','MinimoEstoqueNorm','demanda_diariaNorm','PreçoUnitarioNorm']]

# Padronizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(data_for_clustering)

# Escolher o número de clusters
n_clusters = 5  # Substitua pelo número desejado de clusters

# Aplicar o algoritmo K-Means
kmeans = KMeans(n_clusters=n_clusters)
clusters = kmeans.fit_predict(X)

# Adicionar a coluna de clusters ao DataFrame original
QuantidadeDatas['Cluster'] = clusters
#Separar a lista dos itens

In [ ]:
silhouette_scores = []
for n_clusters in range(2, 16):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(data_for_clustering)
    silhouette_avg = silhouette_score(data_for_clustering, cluster_labels)
    silhouette_scores.append(silhouette_avg)

In [ ]:
plt.plot(range(2, 16), silhouette_scores, marker='o', linestyle='--')
plt.xlabel('Número de Clusters')
plt.ylabel('Coeficiente de Silhueta')
plt.title('Coeficiente de Silhueta para Determinar o Número Ideal de Clusters')
plt.show()

In [ ]:
df.columns

In [ ]:
df[df['nm_material']=='PIN�A ALLIS RETA 15 CM - UNIDADE']

In [ ]:
QuantidadeDatas

In [ ]:
# Suponha que você já tenha o DataFrame "QuantidadeDatas" com a coluna "Cluster" definida

# Agrupe o DataFrame por "Cluster" e calcule as estatísticas desejadas
estatisticas_por_cluster = QuantidadeDatas.groupby('Cluster').agg({
    'CompreMedia': 'mean',  # Média
    'MediaEstoque': 'mean',  # Mediana
    'MinimoEstoque': 'mean',  # Mínimo
    'demanda_diaria': 'mean',
    'PreçoUnitario' : 'mean'
    # Adicione outras colunas e estatísticas desejadas aqui
}).reset_index()

# Calcule a quantidade de itens por cluster
quantidade_itens_por_cluster = QuantidadeDatas['Cluster'].value_counts().reset_index()
quantidade_itens_por_cluster.columns = ['Cluster', 'Quantidade_de_Itens']

# Renomeie as colunas das estatísticas para maior clareza, se necessário
estatisticas_por_cluster = estatisticas_por_cluster.rename(columns={
    'Cluster': 'Cluster',
    'CompreMedia': 'Media_Compra',
    'MediaEstoque': 'Mediana_Estoque',
    'Minimo_Estoque': 'Mínimo_Estoque',
    'PreçoUnitario' : 'PreçoUnitario'
    # Renomeie as colunas conforme necessário
})

# Exiba as estatísticas por cluster e a quantidade de itens por cluster
print("Estatísticas por Cluster:")
print(estatisticas_por_cluster)
print("\nQuantidade de Itens por Cluster:")
print(quantidade_itens_por_cluster)


In [ ]:

ListaComMetricas = []
ListaComMetricas.append(['Item','Valor_Real','Previsão','ErroBIAS','ErroABS','r2'])
#Query = QuantidadeDatas[(QuantidadeDatas['Cluster']==3)&(QuantidadeDatas['Demanda']>QuantidadeDatas['Demanda'].mean())]['Material'].unique()
Query = QuantidadeDatas[(QuantidadeDatas['Cluster']==0)]['Material'].unique()

for i in (Query):

    #Local para colocar o item a analisar
    produtoanalisar = COVID[COVID['nm_material']==i].copy()

    #resumo = produtoanalisar.groupby('dt_geracao')['nr_quantidade'].sum().reset_index()
    #data = resumo.set_index('dt_geracao')

    #Metrica Consumo
    #Consumo = produtoanalisar.groupby('nm_material')['nr_quantidade'].sum()
    #Consumo = Quantidade.reset_index(drop=True)

    #Métrica dias com compra
    #contagem_distintos = COVID.groupby('nm_material')['dt_geracao'].nunique()
    #DistinctDatas = contagem_distintos.reset_index()

    #QuantidadeDatas = pd.merge(Consumo,DistinctDatas,how='inner',on='nm_material')



    #estoque = produtoanalisar.groupby('dt_geracao')['nr_qtd_estoque'].mean().reset_index()
    #estoque = estoque.set_index('dt_geracao')

    produtoanalisar['demanda_diaria'] = -produtoanalisar['nr_qtd_estoque'].diff().fillna(0)
    produtoanalisar['demanda_diaria'] = produtoanalisar['demanda_diaria'].apply(lambda x: max(0, x))
    Demanda = produtoanalisar.groupby('dt_geracao')['demanda_diaria'].sum().reset_index()
    Demanda = Demanda.set_index('dt_geracao')
    data_range = Demanda.index

    #consumo = produtoanalisar.groupby('dt_geracao')['nr_quantidade'].sum().reset_index()
    #consumo = consumo.set_index('dt_geracao')
    
    to_row = int(len(Demanda)*0.5)
    training_data = list(Demanda[0:to_row]['demanda_diaria'])
    testing_data = list(Demanda[to_row:]['demanda_diaria'])
    
    stepwise_fit = auto_arima(Demanda['demanda_diaria'], trace=True,suppress_warnings=True)
    
    p, d, q = stepwise_fit.order
    
    model_prediction = []
    n_test_obser = len(testing_data)
    for j in range(n_test_obser):
        #model = sm.tsa.arima.ARIMA(training_data,order=(1,1,1))
        model = sm.tsa.arima.ARIMA(training_data,order=(p,d,q))
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = list(output)[0]
        model_prediction.append(yhat)
        actual_test_value = testing_data[j]
        training_data.append(actual_test_value)
        #print(output)
        if j == (n_test_obser-1):
            pred = pd.DataFrame(model_prediction,columns=['Previsão'])
            real = pd.DataFrame(testing_data,columns=['Real'])
            result = pd.concat([pred, real], axis=1)
            result['Erro'] = result['Previsão'] - result['Real']
            ErroBIAS = result['Erro'].sum()
            result['Erro_ABS'] = result['Erro'].abs()
            ERROABS = result['Erro_ABS'].sum()
            PrevisãoVar = result['Previsão'].sum()
            ValorRealVar = result['Real'].sum()
            valor_real = result['Real']
            previsao = result['Previsão']
            result['ModeloMedia'] = (Demanda[0:to_row]['demanda_diaria'].mean()*to_row)/len(result)
            result['ErroModeloMedia'] = result['ModeloMedia'] - result['Real']
            result['ErroQuadrado'] = result['Erro']*result['Erro']
            result['ErroModeloMediaQuadrado'] = result['ErroModeloMedia']*result['ErroModeloMedia']
            r2 = (result['ErroModeloMediaQuadrado'].sum() - result['ErroQuadrado'].sum())/result['ErroModeloMediaQuadrado'].sum()
            linha = [i, ValorRealVar, PrevisãoVar, ErroBIAS,ERROABS,r2]
            ListaComMetricas.append(linha)
        else:
            pass


In [ ]:
len(ListaComMetricas)

In [ ]:
Resultados = pd.DataFrame(ListaComMetricas[1:], columns=ListaComMetricas[0])

In [ ]:
Resultados

In [ ]:
ResultadosCluster0 = Resultados

In [ ]:
#ResultadosCluster3 = Resultados

In [ ]:
Resultados['%ValorReal'] = 1-(Resultados['Valor_Real'] - Resultados['Previsão'])/Resultados['Valor_Real']

In [ ]:


# Supondo que Resultados é o seu DataFrame
Resultados.loc[Resultados['%ValorReal'] < -1, '%ValorReal'] = -1
Resultados.loc[Resultados['%ValorReal'] > 1, '%ValorReal'] = 1.1


In [ ]:
Resultados.to_csv('ResultadosPrincipaisMelhorAIC.csv')

In [ ]:
Resultados['%ValorReal'].min()

In [ ]:
Resultados['%ValorReal'].hist(bins=50)

In [ ]:
Resultados[Resultados['%ValorReal']>0.25]

In [ ]:
#ResultadosCluster1 = Resultados

In [ ]:
#ResultadosCluster4 = Resultados

In [ ]:
for cluster_num in range(n_clusters):
    cluster_data = QuantidadeDatas[QuantidadeDatas['Cluster'] == cluster_num]
    plt.scatter(cluster_data['CompreMedia'], cluster_data['MediaEstoque'], label=f'Cluster {cluster_num}')

plt.xlabel('Compra Media')
plt.ylabel('Estoque Medio')
plt.title('Gráfico de Dispersão dos Clusters')
plt.legend()
plt.show()

In [ ]:

for cluster_num in range(n_clusters):
    cluster_data = QuantidadeDatas[QuantidadeDatas['Cluster'] == cluster_num]
    plt.scatter(cluster_data['demanda_diaria'], cluster_data['MediaEstoque'], label=f'Cluster {cluster_num}')

plt.xlabel('demanda_diaria')
plt.ylabel('Estoque Medio')
plt.title('Gráfico de Dispersão dos Clusters')
plt.legend()
plt.show()

In [ ]:
for i in (QuantidadeDatas[QuantidadeDatas['Cluster'] ==0]['Material']):
    tiara = df[df['nm_material']==i].copy()
    tiara['demanda_diaria'] = -tiara['nr_qtd_estoque'].diff().fillna(0)
    tiara['demanda_diaria'] = tiara['demanda_diaria'].apply(lambda x: max(0, x))
    estoque = tiara.groupby('dt_geracao')['nr_qtd_estoque'].mean().reset_index()
    estoque = estoque.set_index('dt_geracao')
    Demanda = tiara.groupby('dt_geracao')['demanda_diaria'].sum().reset_index()
    data_range = Demanda.index
    Demanda = Demanda.set_index('dt_geracao')
    consumo = tiara.groupby('dt_geracao')['nr_quantidade'].sum().reset_index()
    consumo = consumo.set_index('dt_geracao')
    plt.figure(figsize=(10,6))
    plt.grid(True)
    plt.xlabel('Dates')
    plt.ylabel('Quantidade')
    plt.plot(data_range,estoque,color='red',linestyle='dashed',label='Estoque')
    #plt.plot(data_range,data,color='red',label='Compra')
    plt.plot(data_range,Demanda,color='black',label='Demanda',alpha=0.7)
    plt.plot(data_range,consumo,color='green',label='Compra',alpha=0.5)
    #plt.plot(estoque)
    plt.title(i) 
    plt.legend()

In [ ]:
Resultados[(Resultados['ErroBIAS']>(-200))&(Resultados['ErroBIAS']<200)&(Resultados['Valor_Real']>1000)].sort_values(by='ErroABS', ascending=True)

In [ ]:
#Resultados.to_csv('ResultadoAnalisarTableau.csv')

# Previsão de apenas um item

In [ ]:
#Selecionar item
tiara = df[df['nm_material']=='nm_material'].copy()
tiara['demanda_diaria'] = -tiara['nr_qtd_estoque'].diff().fillna(0)
tiara['demanda_diaria'] = tiara['demanda_diaria'].apply(lambda x: max(0, x))
estoque = tiara.groupby('dt_geracao')['nr_qtd_estoque'].mean().reset_index()
Demanda = tiara.groupby('dt_geracao')['demanda_diaria'].sum().reset_index()
data_range = Demanda.index
Demanda = Demanda.set_index('dt_geracao')
consumo = tiara.groupby('dt_geracao')['nr_quantidade'].sum().reset_index()
consumo = consumo.set_index('dt_geracao')
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Quantidade')
plt.plot(Demanda.index,estoque,color='red',linestyle='dashed',label='Estoque')
#plt.plot(data_range,data,color='red',label='Compra')
plt.plot(Demanda.index,Demanda,color='black',label='Demanda',alpha=0.7)
plt.plot(Demanda.index,consumo,color='green',label='Compra',alpha=0.5)
#plt.plot(estoque)
plt.legend()

to_row = int(len(Demanda)*0.5)
training_data = list(Demanda[0:to_row]['demanda_diaria'])
testing_data = list(Demanda[to_row:]['demanda_diaria'])
Demanda[0:to_row]['demanda_diaria'].mean()*to_row
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Demanda')
plt.plot(Demanda[0:to_row]['demanda_diaria'],'green',label='Train data')
plt.plot(Demanda[to_row:]['demanda_diaria'],'blue',label='Test data')
#plt.plot(estoque)
plt.legend()

stepwise_fit = auto_arima(Demanda['demanda_diaria'], trace=True,
suppress_warnings=True)
p, d, q = stepwise_fit.order
model_prediction = []
n_test_obser = len(testing_data)
if p == 0 and d == 0 and q == 0:
    p, d, q = 1, 1, 1
else:
    pass
print(p, d, q)

for i in range(n_test_obser):
    model = sm.tsa.arima.ARIMA(training_data,order=(p,d,q))
    #model = sm.tsa.arima.ARIMA(training_data,order=(1,1,1))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = list(output)[0]
    model_prediction.append(yhat)
    actual_test_value = testing_data[i]
    training_data.append(actual_test_value)
    #print(output)
    
print(model_fit.summary())

plt.figure(figsize=(15,9))
plt.grid(True)

data_range = Demanda[to_row:].index

plt.plot(data_range,model_prediction,color='blue',label='Previsão')
plt.plot(data_range,testing_data,color='red',label='real')
plt.title('Previsão')
plt.xlabel('Data')
plt.ylabel('Consumo')
plt.legend()
plt.show()

pred = pd.DataFrame(model_prediction,columns=['Previsão'])
real = pd.DataFrame(testing_data,columns=['Real'])
result = pd.concat([pred, real], axis=1)
result['Erro'] = result['Previsão'] - result['Real']
#Valor Real
ValorRealVar = result['Real'].sum()
PrevisãoVar = result['Previsão'].sum()
Rmediasimples = Demanda[0:to_row]['demanda_diaria'].mean()*to_row/ValorRealVar
Rprev = PrevisãoVar/ValorRealVar
print('Valor Real')
print(ValorRealVar)
print('Previsão')
print(PrevisãoVar)
print('Erro')
print(result['Erro'].sum())
print('Modelo Mais Simples de Media')

print(Demanda[0:to_row]['demanda_diaria'].mean()*to_row/ValorRealVar)
result['ModeloMedia'] = (Demanda[0:to_row]['demanda_diaria'].mean()*to_row)/len(result)
result['ErroModeloMedia'] = result['ModeloMedia'] - result['Real']
result['ErroQuadrado'] = result['Erro']*result['Erro']
result['ErroModeloMediaQuadrado'] = result['ErroModeloMedia']*result['ErroModeloMedia']


print('R2')
print((result['ErroModeloMediaQuadrado'].sum() - result['ErroQuadrado'].sum())/result['ErroModeloMediaQuadrado'].sum())

In [ ]:
(Demanda[0:to_row]['demanda_diaria'].mean()*to_row)/len(result)

In [ ]:
result['ModeloMedia'] = (Demanda[0:to_row]['demanda_diaria'].mean()*to_row)/len(result)

In [ ]:
result['ErroModeloMedia'] = result['ModeloMedia'] - result['Real']

In [ ]:
result['ErroQuadrado'] = result['Erro']*result['Erro']
result['ErroModeloMediaQuadrado'] = result['ErroModeloMedia']*result['ErroModeloMedia']

In [ ]:
(result['ErroModeloMediaQuadrado'].sum() - result['ErroQuadrado'].sum())/result['ErroModeloMediaQuadrado'].sum()

In [ ]:
COVID_unit = df[(df['ano']==2020)|(df['ano']==2021)]
COVID_unit = COVID_unit.sort_values(by=['nm_material', 'dt_geracao'])
produtoanalisar = COVID_unit[COVID_unit['nm_material']=='ALTEPLASE, 50 MG, P� LI�FILO'].copy()



produtoanalisar['demanda_diaria'] = -produtoanalisar['nr_qtd_estoque'].diff().fillna(0)
produtoanalisar['demanda_diaria'] = produtoanalisar['demanda_diaria'].apply(lambda x: max(0, x))
Demanda = produtoanalisar.groupby('dt_geracao')['demanda_diaria'].sum().reset_index()
Demanda = Demanda.set_index('dt_geracao')
data_range = Demanda.index


In [ ]:
Demanda.plot()

In [ ]:
COVID_unit = df[(df['ano']==2020)|(df['ano']==2021)]
COVID_unit = COVID_unit.sort_values(by=['nm_material', 'dt_geracao'])
produtoanalisar = COVID_unit[COVID_unit['nm_material']=='EQUIPO MACROGOTAS PARA NUTRI��O ENTERAL'].copy()



produtoanalisar['demanda_diaria'] = -produtoanalisar['nr_qtd_estoque'].diff().fillna(0)
produtoanalisar['demanda_diaria'] = produtoanalisar['demanda_diaria'].apply(lambda x: max(0, x))
Demanda = produtoanalisar.groupby('dt_geracao')['demanda_diaria'].sum().reset_index()
Demanda = Demanda.set_index('dt_geracao')
data_range = Demanda.index

    #consumo = produtoanalisar.groupby('dt_geracao')['nr_quantidade'].sum().reset_index()
    #consumo = consumo.set_index('dt_geracao')
    
to_row = int(len(Demanda)*0.5)
training_data = list(Demanda[0:to_row]['demanda_diaria'])
testing_data = list(Demanda[to_row:]['demanda_diaria'])
    
    #stepwise_fit = auto_arima(Demanda['demanda_diaria'], trace=True,suppress_warnings=True)
    
    #p, d, q = stepwise_fit.order
ListaComMetricas_unit = []
ListaComMetricas_unit.append(['Item','Valor_Real','Previsão','ErroBIAS','ErroABS','r2'])    
model_prediction_sing = []


model = sm.tsa.arima.ARIMA(training_data,order=(0,1,1))
#model = sm.tsa.arima.ARIMA(training_data,order=(p,d,q))
model_fit = model.fit()
output = model_fit.forecast()
yhat = list(output)[0]
model_prediction_sing.append(yhat)
actual_test_value = testing_data[j]
training_data.append(actual_test_value)

pred = pd.DataFrame(model_prediction,columns=['Previsão'])
real = pd.DataFrame(testing_data,columns=['Real'])
result = pd.concat([pred, real], axis=1)
result['Erro'] = result['Previsão'] - result['Real']
ErroBIAS = result['Erro'].sum()
result['Erro_ABS'] = result['Erro'].abs()
ERROABS = result['Erro_ABS'].sum()
PrevisãoVar = result['Previsão'].sum()
ValorRealVar = result['Real'].sum()
valor_real = result['Real']
previsao = result['Previsão']



In [ ]:
print(model_fit.summary())

In [ ]:
(sum(training_data)/len(training_data))*to_row

In [ ]:
real.sum()

In [ ]:
result.sum()

In [ ]:
plt.figure(figsize=(15,9))
plt.grid(True)

data_range = Demanda[to_row:].index

plt.plot(data_range,model_prediction_sing,color='blue',marker='o',linestyle='dashed',label='Previsão')
plt.plot(data_range,testing_data,color='red',label='real')
plt.title('Previsão')
plt.xlabel('Data')
plt.ylabel('Consumo')
plt.legend()
plt.show()

In [ ]:
QuantidadeDatas

# Gráfico de Previsão

In [ ]:

ListaComMetricas = []
ListaComMetricas.append(['Item','Valor_Real','Previsão','ErroBIAS','ErroABS','r2'])
#Query = QuantidadeDatas[(QuantidadeDatas['Cluster']==3)&(QuantidadeDatas['Demanda']>QuantidadeDatas['Demanda'].mean())]['Material'].unique()
Query = QuantidadeDatas[(QuantidadeDatas['Cluster']==0)]['Material'].unique()

for i in (Query):

    #Local para colocar o item a analisar
    produtoanalisar = COVID[COVID['nm_material']==i].copy()

    #resumo = produtoanalisar.groupby('dt_geracao')['nr_quantidade'].sum().reset_index()
    #data = resumo.set_index('dt_geracao')

    #Metrica Consumo
    #Consumo = produtoanalisar.groupby('nm_material')['nr_quantidade'].sum()
    #Consumo = Quantidade.reset_index(drop=True)

    #Métrica dias com compra
    #contagem_distintos = COVID.groupby('nm_material')['dt_geracao'].nunique()
    #DistinctDatas = contagem_distintos.reset_index()

    #QuantidadeDatas = pd.merge(Consumo,DistinctDatas,how='inner',on='nm_material')



    #estoque = produtoanalisar.groupby('dt_geracao')['nr_qtd_estoque'].mean().reset_index()
    #estoque = estoque.set_index('dt_geracao')

    produtoanalisar['demanda_diaria'] = -produtoanalisar['nr_qtd_estoque'].diff().fillna(0)
    produtoanalisar['demanda_diaria'] = produtoanalisar['demanda_diaria'].apply(lambda x: max(0, x))
    Demanda = produtoanalisar.groupby('dt_geracao')['demanda_diaria'].sum().reset_index()
    Demanda = Demanda.set_index('dt_geracao')
    data_range = Demanda.index

    #consumo = produtoanalisar.groupby('dt_geracao')['nr_quantidade'].sum().reset_index()
    #consumo = consumo.set_index('dt_geracao')
    
    to_row = int(len(Demanda)*0.5)
    training_data = list(Demanda[0:to_row]['demanda_diaria'])
    testing_data = list(Demanda[to_row:]['demanda_diaria'])
    
    stepwise_fit = auto_arima(Demanda['demanda_diaria'], trace=True,suppress_warnings=True)
    print('aqui1')
    p, d, q = stepwise_fit.order
    print('aqui2')
    model_prediction = []
    n_test_obser = len(testing_data)
    for j in range(n_test_obser):
        #model = sm.tsa.arima.ARIMA(training_data,order=(1,1,1))
        model = sm.tsa.arima.ARIMA(training_data,order=(p,d,q))
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = list(output)[0]
        model_prediction.append(yhat)
        actual_test_value = testing_data[j]
        training_data.append(actual_test_value)
        #print(output)
        if j == (n_test_obser-1):
            pred = pd.DataFrame(model_prediction,columns=['Previsão'])
            real = pd.DataFrame(testing_data,columns=['Real'])
            result = pd.concat([pred, real], axis=1)
            result['Erro'] = result['Previsão'] - result['Real']
            ErroBIAS = result['Erro'].sum()
            result['Erro_ABS'] = result['Erro'].abs()
            ERROABS = result['Erro_ABS'].sum()
            PrevisãoVar = result['Previsão'].sum()
            ValorRealVar = result['Real'].sum()
            valor_real = result['Real']
            previsao = result['Previsão']
            result['ModeloMedia'] = (Demanda[0:to_row]['demanda_diaria'].mean()*to_row)/len(result)
            result['ErroModeloMedia'] = result['ModeloMedia'] - result['Real']
            result['ErroQuadrado'] = result['Erro']*result['Erro']
            result['ErroModeloMediaQuadrado'] = result['ErroModeloMedia']*result['ErroModeloMedia']
            r2 = (result['ErroModeloMediaQuadrado'].sum() - result['ErroQuadrado'].sum())/result['ErroModeloMediaQuadrado'].sum()
            linha = [i, ValorRealVar, PrevisãoVar, ErroBIAS,ERROABS,r2]
            ListaComMetricas.append(linha)
            
            
            print('aqui')
            plt.plot(model_prediction,color='blue',marker='o',linestyle='dashed',label='Previsão')
            plt.plot(testing_data,color='red',label='real')
            plt.title('Previsão')
            plt.xlabel('Data')
            plt.ylabel('Consumo')
            plt.legend()
            plt.show()
        else:
            pass
        
    


In [ ]:
len(data_range)

In [ ]:
len(ListaComMetricas)